Connect to correct port that the board is connected to
find the port like this on terminal:
```
ls /dev/tty.*
```

and send the entire lib folder to it with ampy like this on terminal (change to your path of the board):
```
ampy -p /dev/tty.usbserial-0001 put lib/
```

Make a file called .env with this inside it:

```
#.env
WIFI_SSID=WIFI_NAME
WIFI_PASSWORD=WIFI_PASSWORD
```
change WIFI_NAME and WIFI_PASSWORD to your WiFi credentials

and then run the following on terminal to send it to board (change to your path of the board):

```
ampy -p /dev/tty.usbserial-0001 put .env lib/.env
```

Now connect to the board on this notebook by running the next code, change path to your board port:

In [3]:
%serialconnect to --port="/dev/tty.usbserial-0001" --baud=115200

Connecting to --port=/dev/tty.usbserial-0001 --baud=115200 
Ready.


You can test if it is connecting to wifi here:

In [4]:
from lib.utils import connect_wifi, load_env_vars

env_vars = load_env_vars("lib/.env")
# connect WiFI
ssid = env_vars.get("WIFI_SSID")
password = env_vars.get("WIFI_PASSWORD")
connect_wifi(ssid, password)

connecting to network...
network config: ('192.168.0.76', '255.255.255.0', '192.168.0.1', '194.168.4.100')


Now you need to run the flask server by running on terminal:
```
python /Users/rishil/Desktop/FYP/EEG-decoding/eeg_lib/simple_server.py
```

Inside the file you can change your output logs file name:
```
DEFAULT_FILENAME = "/Users/rishil/Desktop/FYP/EEG-decoding/eeg_lib/logs/rishil_test_1_raw.json"
```

You will need to send posts to the server with the code as followed, you can run it here to test:

In [16]:
import gc
from micropython import alloc_emergency_exception_buf
from lib.requests import MicroWebCli as requests
from lib.utils import load_env_vars
import ujson as json
from lib.utils import connect_wifi, load_env_vars

# allocate exception buffer for ISRs
alloc_emergency_exception_buf(100)
# enable and configure garbage collection
gc.enable()
gc.collect()
gc.threshold(gc.mem_free() // 4 + gc.mem_alloc())

from machine import Pin
import time

p26 = Pin(26, Pin.OUT)  #green
p13 = Pin(13, Pin.OUT)  #red
buttonA = Pin(32, Pin.IN)
buttonB = Pin(34, Pin.IN)
p26.on()
p13.on()

from ulab import numpy as np
import utime as time
from lib.runner import Runner  

runner = Runner('CCA', buffer_size=256)
runner.setup()
runner.run()

p26.off()
p13.off()

print(2)
time.sleep(5)

p26.on()
p13.on()

env_vars = load_env_vars("lib/.env")
ssid = env_vars.get("WIFI_SSID")
password = env_vars.get("WIFI_PASSWORD")
connect_wifi(ssid, password)

while True:
    time.sleep(4)
    data = runner.periph_manager.read_adc_buffer()
    toSend = {"raw_data":data}
    print(toSend)
    requests.JSONRequest("http://192.168.0.13:5001/", toSend)
    del data

ADC initialised
SPI initialised
DigiPot set to 250 = gain of 2.031451
2
.network config: ('192.168.0.76', '255.255.255.0', '192.168.0.1', '194.168.4.100')
.{'raw_data': [2037, 2075, 1987, 1860, 1934, 2058, 2064, 1973, 1851, 1770, 1939, 1785, 1810, 1979, 2089, 2037, 1927, 1808, 1775, 2086, 2026, 1904, 1791, 1776, 1904, 2042, 2068, 1771, 1794, 1967, 2078, 2055, 1952, 1826, 2032, 1949, 1817, 1765, 1854, 2007, 2088, 2018, 1771, 1906, 2038, 2071, 1986, 1857, 1763, 2064, 2058, 1951, 1834, 1771, 1843, 2003, 1942, 1818, 1772, 1875, 2079, 1927, 1799, 1767, 2078, 2000, 1872, 1772, 1778, 1939, 2064, 1862, 1763, 1794, 1958, 2072, 2034, 1929, 1806, 1995, 2080, 2006, 1887, 1771, 1771, 1931, 1850, 1774, 1919, 2043, 2063, 1968, 1840, 1762, 2083, 2034, 1920, 1799, 1776, 1930, 2063, 1858, 1762, 1808, 1973, 2080, 2032, 1923, 1827, 1990, 2083, 1936, 1771, 1903, 2033, 2069, 1767, 1792, 1968, 2078, 2057, 1946, 1822, 1990, 2090, 2026, 1907, 1787, 1776, 1914, 2000, 1877, 1770, 1787, 1946, 2067, 2053, 1915, 20

.

*** Sending Ctrl-C



Traceback (most recent call last):
  File "<stdin>", line 52, in <module>
  File "lib/requests.py", line 163, in JSONRequest
  File "lib/requests.py", line 381, in OpenRequestJSONData
  File "lib/requests.py", line 262, in OpenRequest
  File "lib/requests.py", line 445, in URL
  File "lib/requests.py", line 86, in _urlEncode
  File "lib/requests.py", line 79, in _quote
KeyboardInterrupt: 


In [17]:
runner.stop()

Now if it works on here then you need to make a file called main.py with that code in and send it to the board using:

Make sure source main.py is in same directory as this notebook

In [ ]:
%sendtofile main.py --source main.py